In [1]:
import numpy as np
import pandas as pd
import scipy
import multiprocessing
import matplotlib.pyplot as plt
import os
import subprocess
from tqdm import tqdm

Constants and functions for generating and computing NEOMOD3 orbits and their impact probabilities:

In [2]:
# constants for earth and sun
a0 = 1.  # semi-major axis of earth in au
m = 5.97e24  # mass of earth in kg
R = 4.259e-5  # radius of earth in au
M = 1.988e30  # mass of sun in kg
e0 = 0.01671123  # eccentricitiy of earth

compiler = 'gfortran'  # name of fortran compiler to use
# compile fortran code if it hasn't been already
subprocess.run([compiler, './Pokorny_et_al_2013/PROB_KOZAI_V001.f90', '-o', './Pokorny_et_al_2013/CODE', '-O3'])

# define function
def get_pokorny_annual_P_packed(aeiperi, a_earth=a0, e_earth=e0, R_earth=R):
    """
    Get the collisional probability using Petr Pokorny's 2013 code with a, e, i, peri packed as a tuple
    Takes in a given a, e, i and argument of pericenter of the projectile and returns the annual collisional probability

    a is in units of AU, i is in units of degrees, argument of pericenter is in degrees
    """
    a, e, i, peri = aeiperi  # unpack
    # run fortran code
    output = subprocess.run(['./Pokorny_et_al_2013/CODE', '<'], 
                            input=f'{a},{e},{i},{peri}\n{a_earth}\n{e_earth}'.encode('utf-8'), 
                            capture_output=True)
    # print(output)
    col_prob = np.float64(output.stdout.decode('utf-8').split()[-1]) * (R_earth ** 2)
    return col_prob

# compile the NEOMOD3 model
neomod3_filename = './NEOMOD3_CODE'
# compile neomod3 fortran code
subprocess.run([compiler, './neomod3_simulator.f', '-o', neomod3_filename, '-O3'])

# generate debiased steady-state orbital distribution for both
size_min = 0.0079  # minimum size NEOMOD 3, km
size_max = 0.016  # maximum size NEOMOD 3, km
n_objs = 10000  # number of objects to generate
seed = -50202002  # seed to use

Perform the generation and IP calculation:

In [3]:
# generate neomod3 output:
# Returns H, a (AU), e, i (degrees), diameter (km), albedo
neomod3_output = np.array(subprocess.run([neomod3_filename, '<'], 
                                input=f'input_neomod3.dat\n{seed}\n{n_objs}\n{size_min} {size_max}'.encode('utf-8'), 
                                capture_output=True).stdout.decode('utf-8').split(), dtype=np.float64).reshape(n_objs, 6)  # 6 parameters

# unpack values
neomod3_h, neomod3_a, neomod3_e, neomod3_i, neomod3_size, neomod3_alb = neomod3_output.T
# generate random arguments of pericenter
random_peris = np.random.uniform(low=0., high=360., size=n_objs)  # pokorny annual P takes in degrees

# compute impact probabilities:
neomod3_aeiperis = np.c_[neomod3_a, neomod3_e, neomod3_i, random_peris]
# multiprocess, computing for neomod2 and 3
pool = multiprocessing.Pool()
neomod3_pts = np.array(list(tqdm(pool.imap(get_pokorny_annual_P_packed, neomod3_aeiperis), total = len(neomod3_aeiperis))))
# join and close
pool.close()
pool.join()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:16<00:00, 130.17it/s]


In [4]:
neomod3_output, neomod3_pts

(array([[2.6782e+01, 1.6610e+00, 3.7840e-01, 9.3000e+00, 9.6000e-03,
         3.7500e-01],
        [2.6575e+01, 2.3040e+00, 5.5390e-01, 5.9300e+00, 1.1100e-02,
         3.3750e-01],
        [2.6082e+01, 9.2100e-01, 5.4010e-01, 2.4050e+01, 1.1100e-02,
         5.2600e-01],
        ...,
        [2.9835e+01, 1.3460e+00, 3.9680e-01, 4.4600e+00, 1.1600e-02,
         1.5300e-02],
        [2.7296e+01, 2.4060e+00, 7.0410e-01, 6.8800e+00, 1.1200e-02,
         1.6890e-01],
        [2.7793e+01, 1.6550e+00, 6.9760e-01, 7.9700e+00, 7.9000e-03,
         2.1540e-01]]),
 array([0.00000000e+00, 0.00000000e+00, 1.89186615e-09, ...,
        4.23293861e-09, 1.33582756e-09, 2.00313773e-09]))

NEOMOD3 impact energies computed using the actual (simulated) velocity at time of impact rather than a fixed 20 km/s